# Introduction to Basic Reservoir Engineering simulations with OpenFOAM

## Convection-Diffustion Case

### 1. Imports and initial configuration

We make sure plots are "inlined", import an improved `DataFrame` class and prepare the case directory for processing

In [1]:
%matplotlib inline
from matplotlib import pyplot as plt
from PyFoam.Wrappers.Pandas import PyFoamDataFrame as DataFrame
from PyFoam.IPythonHelpers import storage
store=storage()

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Then, we import the `Case` class and instantiate our case object

In [2]:
from PyFoam.IPythonHelpers.Case import Case
# Pass the absolute path to the case to Case class
case=Case('/home/linux1/foam/linux1-4.0/run/intro')

/usr/lib/python2.7/site-packages/IPython/html.py:14: ShimWarning: The `IPython.html` package has been deprecated since IPython 4.0. You should import from `notebook` instead. `IPython.html.widgets` has moved to `ipywidgets`.
  "`IPython.html.widgets` has moved to `ipywidgets`.", ShimWarning)


### 2. Investigating the case

Retrieve some mesh statistics (Use `case.<TAB>` to learn more) as HTML objects

In [3]:
# First, make sure the mesh is there
!blockMesh > /dev/null
case.size()

Faces,46
Points,40
Cells,9


In [4]:
case.boundaryConditions()

<!-- -->,inlet,noFlow,outlet
Patch Type,patch,empty,patch
Length,1,36,1
T,fixedValue,empty,fixedValue
U,fixedValue,empty,fixedValue


You can even verify fields' dimensions here 

In [5]:
case.dimensions()

Name,kg,m,s,K,mol,A,cd
T,0,0,0,1,0,0,0
U,0,1,-1,0,0,0,0


and also the variables values at cell centers

In [6]:
case.internalField()

Name,Value
T,uniform 0
U,uniform (0.03 0 0)


Finally, the numerical solver's settings

In [7]:
case.linearSolvers()

Name,Solver,Abs. Tolerance,Relative Tol.
T,smoothSolver,0.001,0


### Preparing and running the case

In [8]:
# Use PyFoam as a library to launch simulations
from PyFoam.Applications.Runner import Runner

# On commandline, you can run:
#! pyFoamRunner.py --clear --progress scalarTransportFoam -case .

## You should notice that
# PyFoam application options start with --
# OpenFOAM solver options start with a single -

# Clear the case, show only the progress, plot all plots
# Choose scalarTransportFoam, and provide case absolute path as its option
rn = Runner(args=["--clear","--progress","scalarTransportFoam","-case",case.path])


Clearing out old timesteps ....
 t =          2

 PyFoam WARNING on line 142 of file /home/linux1/foam/foam-extend-4.0/ThirdParty/packages/PyFoam-0.6.4/platforms/noarch/lib/python2.7/site-packages/PyFoam/LogAnalysis/GeneralLineAnalyzer.py : Problem doing timelines could not convert string to float:  


### Investigating the run

In [9]:
# Get simulation Data as a Python dictionary
data = rn.getData()

In [10]:
# Discover what info is stored
data.keys()

['stepNr',
 'casefullname',
 'cpuTime',
 'fatalStackdump',
 'fatalFPE',
 'logfile',
 'commandLine',
 'hostname',
 'solverFull',
 'lasttimesteptime',
 'cpuUserTime',
 'cpuSystemTime',
 'analyzed',
 'endtime',
 'remark',
 'OK',
 'casename',
 'solver',
 'lines',
 'interrupted',
 'fatalError',
 'usedMemory',
 'starttime',
 'time',
 'wallTime']

In [11]:
# Has the run finished OK?
rn.OK

True

In [12]:
# If failed, is it an OpenFOAM Fatal Error?
rn.fatalError

False

In [13]:
# Or did the run get interrupted
rn.interrupted

False

In [14]:
# Was there a run "remark" (For over-the-network simulations)
rn.remark

'No remark given'

In [15]:
# CPU TIME 
rn.cpuSystemTime, rn.cpuUserTime, rn.cpuTime

(0.0, 0.0, 0.0)

In [16]:
# Newest time directory
case.sol.latestDir()

'/home/linux1/foam/linux1-4.0/run/intro/2'

### The list of T values

What we *really* want to extract in this case is the `nonuniform list of T` at time = 2 (second timeStep)!

In [17]:
case.internalField(time=2.0)
# But this is HTML!!

Name,Value
T,nonuniform List<scalar> 9...
U,uniform (0.03 0 0)
phi,uniform 0.0003


In [18]:
# The easiest way, for this case is to directly parse 2/T file

# Import parsedFile class as sf
from PyFoam.RunDictionary.ParsedParameterFile import ParsedParameterFile as sf

# The 2/T file representation
T = sf(case.sol.latestDir()+'/T')

# What content the file has?
print("2/T content: %s\n" % T.content.keys())

# Extract the list of values
print("T at cell center: %s" % T.content['internalField'].value())

2/T content: ['dimensions', 'internalField', 'boundaryField']

T at cell center: [0.98908405, 0.95972003, 0.92028125, 0.8673079, 0.79608295, 0.70017418, 0.57081901, 0.39609872, 0.15982931]
